In [1]:
import numpy as np
import pandas as pd
import zipfile
import dask.dataframe as dk
import tensorflow as tf
import io
import os
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
# temp_dir = "C:/Users/hoang/FileCSV_DACN_2025/ddos_dos_"

input_files = [f"file{i+1}.csv" for i in range(3)]
temp_dir =  ["/mnt/c/Users/hoang/FileCSV_DACN_2025/", "C:/Users/hoang/FileCSV_DACN_2025/"]

if os.name=='nt':
    temp_dir=temp_dir[1]
else:
    temp_dir= temp_dir[0]

input_files = [temp_dir + output_file for output_file in input_files]
print(input_files)
df = [dk.read_csv(file) for file in input_files]
for index in range(len(df)):
    df[index]= df[index].drop(columns='label')
# test_df = dk.read_csv("FL_Dataset/test.csv")

# input_zip = "/mnt/c/Users/hoang/FileCSV_DACN_2025/2Type.zip"
# csv_files = []
# with zipfile.ZipFile(input_zip, 'r') as z:
#     csv_files = [f for f in z.namelist() if f.endswith('.csv')]
# print(csv_files)
# df = [dk.read_csv(f'zip://{file}::{input_zip}') for file in csv_files]
# print(df[0].shape)

print(df[1].dtypes)

2025-05-23 08:34:12.185737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747989252.327262   15162 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747989252.367551   15162 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747989252.692965   15162 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747989252.693032   15162 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747989252.693034   15162 computation_placer.cc:177] computation placer alr

['/mnt/c/Users/hoang/FileCSV_DACN_2025/file1.csv', '/mnt/c/Users/hoang/FileCSV_DACN_2025/file2.csv', '/mnt/c/Users/hoang/FileCSV_DACN_2025/file3.csv']
id.orig_h         float64
id.orig_p         float64
id.resp_h         float64
id.resp_p         float64
duration          float64
orig_bytes        float64
resp_bytes        float64
conn_state        float64
missed_bytes      float64
history           float64
orig_pkts         float64
orig_ip_bytes     float64
resp_pkts         float64
resp_ip_bytes     float64
detailed-label      int64
proto_icmp          int64
proto_tcp           int64
proto_udp           int64
service_dhcp        int64
service_dns         int64
service_http        int64
service_irc         int64
service_ssh         int64
service_ssl         int64
dtype: object


In [2]:
batch_size = 512
ratio_test_all = 0.2

from dask_ml.model_selection import train_test_split 
# Split 
# train_df, val_test_df = train_test_split(df, test_size=0.20, random_state=42)
# val_df, test_df = train_test_split(val_test_df, test_size=0.75, random_state=42)

# # load từng batch
features_len = len(df[1].columns)-1
print("Feature Len: ",features_len)
def dask_to_tf_dataset(dask_df, batch_size, num_classes): 
    def generator():
        for batch in dask_df.to_delayed():
            batch=batch.compute()  
            if batch.empty:
                continue

            X = batch.drop(columns='detailed-label').values.astype(np.float32)
            y = batch['detailed-label'].values
            y_onehot = to_categorical(y, num_classes=num_classes)  

            num_splits = max(1, len(X) // batch_size)  # Đảm bảo không chia nhỏ quá mức
            X_batches = np.array_split(X, num_splits)
            y_batches = np.array_split(y_onehot, num_splits)

            for X_batch, y_batch in zip(X_batches, y_batches):
                yield X_batch, y_batch
                
    output_signature = ( 
        tf.TensorSpec(shape=(None, features_len), dtype=tf.float32), 
        tf.TensorSpec(shape=(None, 3), dtype=tf.int32),
    )
    
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)

Feature Len:  23


In [3]:
# train_df1, test_df1 = df1.random_split([1 - ratio_test_all, ratio_test_all])
# train_df2, test_df2 = df2.random_split([1 - ratio_test_all, ratio_test_all])
# train_df3, test_df3 = df3.random_split([1 - ratio_test_all, ratio_test_all])
batch_size = 512
train_dfs = []
val_dfs = []
test_dfs= []
for dff in df:
    train_df, val_test_df =dff.random_split([1 - ratio_test_all, ratio_test_all])
    test_df, val_df = val_test_df.random_split([1-0.25, 0.25])
    train_dfs.append(train_df)
    val_dfs.append(val_df)
    test_dfs.append(test_df)
   

train_gens = [dask_to_tf_dataset(train_df, batch_size=batch_size,num_classes=3).repeat() for train_df in train_dfs]
val_gens = [dask_to_tf_dataset(val_df , batch_size=batch_size, num_classes=3).repeat() for val_df in val_dfs]
test_gens = [dask_to_tf_dataset(test_df , batch_size=batch_size, num_classes=3).repeat() for test_df in test_dfs]

I0000 00:00:1747989259.056001   15162 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 7.5


In [4]:
import datetime
import numpy as np
import tenseal as ts
#
from server import Server
from client import Client
num_servers = 1
num_clients = 3

stepsPerEpoch_Clients = [int( np.ceil(train_dfs[index].shape[0])/batch_size) for index in range(num_clients)] #[40, 40, 40] 
stepsValidate_Clients =  [int( np.ceil(val_dfs[index].shape[0])/batch_size) for index in range(num_clients)] #[5,5,5]
stepsTest_Clients =[ int(np.ceil(test_dfs[index].shape[0])/batch_size) for index in range(num_clients)]  # [10,10,10] 
print(stepsPerEpoch_Clients)
print(stepsValidate_Clients)
print(stepsTest_Clients)
# int(np.ceil(test_df.shape[0]))
active_servers_list  = ['server_'+str(i)\
                        for i in range(num_servers)]
active_clients_list  = ['client_'+str(i)\
                        for i in range(num_clients)]

[22193, 44386, 66573]
[1390, 2772, 4164]
[4156, 8313, 12479]


In [ ]:
print(active_servers_list)
print(active_clients_list)

def init_he_context():
    """Thiết lập context mã hóa đồng hình"""
    context = ts.context(
        ts.SCHEME_TYPE.CKKS, # ckks cho số thực, bfv cho int
        poly_modulus_degree= 32768,   #8192,
        coeff_mod_bit_sizes=[60, 40,40, 40, 60]
    )
    context.generate_galois_keys()
    context.global_scale = 2**40
    return context

context = init_he_context()
agents_dict= {}
serverObjects={}
clientObjects={}
serverObjects = {server_name: Server(server_name=server_name, \
                        active_clients_list=active_clients_list) \
                        for server_name in active_servers_list}
print(type(serverObjects))
clientObjects = {client_name: Client(client_name, train_gens[clientID], val_gens[clientID], test_gens[clientID], \
                        stepsPerEpoch_Clients[clientID],stepsValidate_Clients[clientID], stepsTest_Clients[clientID],\
                        active_clients_list = active_clients_list, he_context=context) \
                        for clientID, client_name in enumerate(active_clients_list)}

# for index, client_name in enumerate(active_clients_list):
#     clientObjects[client_name].set_steps_per_epoch(stepsPerEpoch_Clients[index])
#     clientObjects[client_name].get_steps_per_epoch()
#     clientObjects[client_name].set_validation_steps(stepsValidate_Clients[index])
#     clientObjects[client_name].get_validation_steps()
#     clientObjects[client_name].set_test_steps(stepsTest_Clients)
#     clientObjects[client_name].get_test_steps()
    
# lưu dict
agents_dict['server'] = serverObjects
agents_dict['client'] = clientObjects

# init agents_dict vừa tạo vào client, server
for agent_name, agent in serverObjects.items():
    agent.set_agentsDict(agents_dict=agents_dict)
for agent_name, agent in clientObjects.items():
    agent.set_agentsDict(agents_dict=agents_dict)

client_name = 'client_1'
print("Agent_Dict: ", agents_dict['client'][client_name])

server = agents_dict['server']['server_0']
print(server)

['server_0']
['client_0', 'client_1', 'client_2']
<class 'dict'>
Agent_Dict:  <client.Client object at 0x7fc9ff7b93a0>


: 

In [ ]:
if __name__ == '__main__':
    server.InitLoop()
    server.final_statistics()

====================================== Đang chạy Iteration 1 ======================================
Epoch 1/4
Epoch 1/4
Epoch 1/4


I0000 00:00:1747989482.750180   15254 service.cc:152] XLA service 0x7fc938008610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747989482.751106   15254 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 with Max-Q Design, Compute Capability 7.5
2025-05-23 08:38:03.005983: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747989483.477930   15256 cuda_dnn.cc:529] Loaded cuDNN version 90300


    2/44386 ━━━━━━━━━━━━━━━━━━━━ 54:18 73ms/step - accuracy: 0.4045 - loss: 1.3782   

I0000 00:00:1747989489.376265   15256 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  177/22193 ━━━━━━━━━━━━━━━━━━━━ 9:28 26ms/step - accuracy: 0.7453 - loss: 0.6918

...

  181/22193 ━━━━━━━━━━━━━━━━━━━━ 10:07 28ms/step - accuracy: 0.7459 - loss: 0.6909

2025-05-23 08:38:14.329078: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


22434/44386 ━━━━━━━━━━━━━━━━━━━━ 9:02 25ms/step - accuracy: 0.8370 - loss: 0.4605

2025-05-23 08:47:23.609408: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng20{k2=0,k4=1,k5=1,k6=0,k7=0,k19=0} for conv %cudnn-conv-bias-activation.7 = (f32[515,128,1,23]{3,2,1,0}, u8[0]{0}) custom-call(f32[515,128,1,23]{3,2,1,0} %bitcast.728, f32[128,128,1,3]{3,2,1,0} %bitcast.732, f32[128]{0} %bitcast.734), window={size=1x3 pad=0_0x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv1d_1_2/convolution" source_file="/home/hoangvn/miniconda3/miniconda3/envs/doan/lib/python3.9/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false} is taking a while...
2025-05-23 08:47:23.612269: E external/local_xla/xla/service/slow_operation_alarm.

22193/22193 ━━━━━━━━━━━━━━━━━━━━ 623s 27ms/step - accuracy: 0.7672 - loss: 0.6153 - val_accuracy: 0.6978 - val_loss: 0.8212
Epoch 2/4
22193/22193 ━━━━━━━━━━━━━━━━━━━━ 576s 26ms/step - accuracy: 0.6971 - loss: 0.8034 - val_accuracy: 0.4124 - val_loss: 2.5930
Epoch 3/4
44386/44386 ━━━━━━━━━━━━━━━━━━━━ 1201s 27ms/step - accuracy: 0.8078 - loss: 0.5245 - val_accuracy: 0.7539 - val_loss: 0.9314
Epoch 2/4
22431/44386 ━━━━━━━━━━━━━━━━━━━━ 8:04 22ms/step - accuracy: 0.7834 - loss: 0.5690

In [ ]:
from tensorflow.keras.models import load_model
# tempdirs = ["D:/DoAnChuyenNganh_Train/client_0_log/11h18p__02-04-2025/", "D:/DoAnChuyenNganh_Train/client_1_log/11h18p__02-04-2025/",  "D:/DoAnChuyenNganh_Train/client_2_log/11h18p__02-04-2025/"]
timeFolder="14h24p__15-04/"
tempdirs = [f"D:/DoAnChuyenNganh_Train/log/client_{i}_log/" for i in range(len(active_clients_list))]


model_names =[timeFolder+f"model_{i+1}.keras" for i in range(5)]
print(model_names)
models = {}

for i, client_name in enumerate(active_clients_list):
    models[client_name] = [load_model(tempdirs[i]+model_name) for model_name in model_names]
print (models['client_0'])

['14h24p__15-04/model_1.keras', '14h24p__15-04/model_2.keras', '14h24p__15-04/model_3.keras', '14h24p__15-04/model_4.keras', '14h24p__15-04/model_5.keras']
[<Sequential name=sequential, built=True>, <Sequential name=sequential, built=True>, <Sequential name=sequential, built=True>, <Sequential name=sequential, built=True>, <Sequential name=sequential, built=True>]


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.models import load_model

import numpy as np

num_batch_test_eachClient  =  []
for index, test_df in enumerate(test_dfs):
    num_samples_test = test_df.shape[0].compute()
    # Tính số batch
    num_batches_test = int(np.ceil(num_samples_test / batch_size))
    num_batch_test_eachClient.append(num_batches_test)

print("Num Batch Each Client: ", num_batch_test_eachClient)
X_tests = {}
Y_tests = {}
Y_preds= {}
for i, client_name in enumerate(active_clients_list):
    Y_tests[client_name]={}
    Y_preds[client_name]={}

# for i, client_name in enumerate(active_clients_list):
#     X_test = []
#     y_test = []
#     for X_batch, y_batch in test_gens[i].take(num_batch_test_eachClient[i]):
#         X_test.extend(X_batch.numpy().flatten())
#         y_test.extend(y_batch.numpy().flatten())
#         y_pred = []
#         for iteration in range(5):    
#     # .as_numpy_iterator():
#         # # take(12000):
#         # X_test_list.append(X_batch.numpy())
#         # y_test_list.append(y_batch.numpy())  # .numpy()

#         # # Gộp tất cả batch lại
#         # X_test = np.concatenate(X_test_list, axis=0)
#         # y_test = np.concatenate(y_test_list, axis=0)

#         # # Nếu y_test đang ở dạng one-hot, chuyển về dạng số
#         # y_test = np.argmax(y_test, axis=1)
#             y_pred_pre = models[client_name][iteration].predict(X_batch, verbose=0)
#             y_pred.extend((y_pred_pre > 0.5).astype(int).flatten())
    
#         Y_tests[client_name][iteration] = y_test
#         Y_preds[client_name][iteration] = y_pred
    
# print(Y_preds)

Num Batch Each Client:  [4160, 8324, 12488]


In [ ]:
# Client 0
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

precisions = []
recalls = []
f1s = []

Iterations = [f"Iteration {index+1}" for index in range(5)]

X_test = []
y_test = []
for X_batch, y_batch in test_gens[i].take(num_batch_test_eachClient[0]):
    X_test.append(X_batch.numpy())
    y_test.append(y_batch.numpy())
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

print(len(X_test))
for iteration in range(len(Iterations)):
    print(models[client_name][iteration])
    y_pred_pre = models[client_name][iteration].predict(X_test, verbose=1)
    y_pred = (y_pred_pre > 0.5).astype(int).flatten()
    
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    recalls.append(recall_score(y_test, y_pred, average='binary'))
    f1s.append(f1_score(y_test, y_pred, average='binary'))


2136424
<Sequential name=sequential, built=True>


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 320, but received input with shape (32, 128)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 23), dtype=float32)
  • training=False
  • mask=None

In [ ]:
print("Precision Score: ", precisions)
print("Recall Score: ", recalls)
print("F1 Score: ", f1s)  


# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 0")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()


attack_types = ['BenignTraffic', 'DoS&DDoS']
cm = confusion_matrix(y_test, y_pred)
    # Vẽ heatmap
plt.figure(figsize=(12, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=attack_types, yticklabels=attack_types)
plt.yticks(rotation=360)
# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 0")

# Hiển thị
plt.show()


# attack_types = ['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
#                    'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']

metrics = []

# Số lượng lớp (10 lớp)
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(12, 5), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 0")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client 1
import matplotlib.pyplot as plt

precisions = []
recalls = []
f1s = []
y_pred = []

Iterations = [f"Iteration {index+1}" for index in range(5)]

X_test = []
y_test = []

print(num_batch_test_eachClient[1])

for X_batch, y_batch in test_gens[1].take(num_batch_test_eachClient[1]):
    X_test.append(X_batch.numpy())
    y_test.append(y_batch.numpy())
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

for iteration in range(len(Iterations)):
    y_pred_pre = models['client_1'][iteration].predict(X_test, verbose=1)
    y_pred = (y_pred_pre > 0.5).astype(int).flatten()
    
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    recalls.append(recall_score(y_test, y_pred, average='binary'))
    f1s.append(f1_score(y_test, y_pred, average='binary'))
    
print("Precision Score: ", precisions)
print("Recall Score: ", recalls)
print("F1 Score: ", f1s)  


# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 1")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()


attack_types = ['BenignTraffic', 'DoS&DDoS']
cm = confusion_matrix(y_test, y_pred)
    # Vẽ heatmap
plt.figure(figsize=(12, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=attack_types, yticklabels=attack_types)
plt.yticks(rotation=360)
# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 1")

# Hiển thị
plt.show()

# attack_types = ['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
#                    'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']

metrics = []

# Số lượng lớp (10 lớp)
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(12, 5), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 1")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client 2
import matplotlib.pyplot as plt

precisions = []
recalls = []
f1s = []

y_pred = []
Iterations = [f"Iteration {index+1}" for index in range(5)]

X_test = []
y_test = []
for X_batch, y_batch in test_gens[2].take(num_batch_test_eachClient[2]):
    X_test.append(X_batch.numpy())
    y_test.append(y_batch.numpy())
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

for iteration in range(len(Iterations)):
    y_pred_pre = models['client_2'][iteration].predict(X_test, verbose=1)
    y_pred = (y_pred_pre > 0.5).astype(int).flatten()
    
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    recalls.append(recall_score(y_test, y_pred, average='binary'))
    f1s.append(f1_score(y_test, y_pred, average='binary'))
    
print("Precision Score: ", precisions)
print("Recall Score: ", recalls)
print("F1 Score: ", f1s)  


# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(Iterations, precisions, marker='o', linestyle='-', label="Precision", color='blue')
plt.plot(Iterations, recalls, marker='s', linestyle='-', label="Recall", color='red')
plt.plot(Iterations, f1s, marker='^', linestyle='-', label="F1-score", color='green')

# Thêm tiêu đề và nhãn
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Precision, Recall, F1-score over different iterations _ CLIENT 2")
plt.legend()
plt.ylim(0, 1)  # Giới hạn từ 0 đến 1
plt.grid(True)

# Hiển thị đồ thị
plt.show()


attack_types = ['BenignTraffic', 'DoS&DDoS']
cm = confusion_matrix(y_test, y_pred)
    # Vẽ heatmap
plt.figure(figsize=(12, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=attack_types, yticklabels=attack_types)
plt.yticks(rotation=360)
# Thêm nhãn
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CLIENT 2")

# Hiển thị
plt.show()


# attack_types = ['BenignTraffic', 'DDoS-ICMP_Flood', 'DDoS-PSHACK_Flood', 'DDoS-RSTFINFlood', 'DDoS-SYN_Flood', 
#                    'DDoS-SynonymousIP_Flood', 'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'DoS-SYN_Flood', 'DoS-TCP_Flood', 'DoS-UDP_Flood']

metrics = []

# Số lượng lớp (10 lớp)
num_classes = len(attack_types)

# Duyệt từng lớp để tính TP, FP, TN, FN
for i in range(num_classes):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    
    metrics.append([attack_types[i], TP, FP, TN, FN])

# Chuyển thành DataFrame
df_metrics = pd.DataFrame(metrics, columns=["Attack_Types", "TP", "FP", "TN", "FN"])

# Vẽ biểu đồ
df_metrics.set_index("Attack_Types").plot(kind="bar", figsize=(12, 5), colormap="viridis")

# Thêm nhãn
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("TP, FP, TN, FN for Each Class CLIENT 2")
plt.xticks(rotation=45)
plt.legend(["TP", "FP", "TN", "FN"])

# Hiển thị
plt.show()

In [ ]:
# Client_0 
import matplotlib.pyplot as plt
import numpy as np

file_dir= "log/client_0_log/14h24p__15-04/"
file_names = [f"Iteration_{index+1}.csv" for index in range(5)]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(18, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

In [ ]:
# Client_1
import matplotlib.pyplot as plt
import numpy as np

file_dir= "log/client_1_log/14h24p__15-04/"
file_names = [f"Iteration_{index+1}.csv" for index in range(5)]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(18, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs Client 1")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()

In [ ]:
# Client_2
import matplotlib.pyplot as plt
import numpy as np

file_dir= "log/client_2_log/14h24p__15-04/"
file_names = [f"Iteration_{index+1}.csv" for index in range(5)]

import pandas as pd
import matplotlib.pyplot as plt

accuracy= []
loss = []
epochs = ["Epoch_"+str(i) for i in range(5)]
for i, file_name in enumerate(file_names):
    # Đọc file CSV
    df = pd.read_csv(file_dir + file_name)  # Đổi tên file nếu cần

    # Chuyển "NA" thành NaN và xử lý nếu cần
    df.replace("NA", None, inplace=True)

      # Đảm bảo epoch là số nguyên
    df["accuracy"] = df["accuracy"].astype(float)
    df["loss"] = df["loss"].astype(float)
    accuracy.append(df["accuracy"])
    loss.append(df["loss"])
    print("Iteration "+str(i)+": ")
    for epoch_index,  epoch in enumerate(epochs):
        print(epoch + f": Accuracy: {df['accuracy'][epoch_index]} \t Loss: {df['loss'][epoch_index]}")

plt.figure(figsize=(18, 6))

# Vẽ Accuracy
plt.subplot(1,2,1)
for i in range(3):
    plt.plot(epochs, accuracy[i], marker="o", linestyle="-", label=f"Iteration {i+1}")
    
plt.title("Accuracy over Epochs _Client 2")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Vẽ Loss
plt.subplot(1, 2, 2)
for i in range(3):
    plt.plot(epochs, loss[i], marker="s", linestyle="-", label=f"Iteration {i+1}")

plt.title("Loss over Epochs Client 2")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()